In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

def load_file(file_path: str) -> dict:
    """
    從劉子揚生成的各種 instance 檔案中讀取資料 \\
    param: file_path: 檔案路徑，直接放例如 "instance0_v1.xlsx" 就可以了 \\
    return: 讀取的資料
    """

    import pandas as pd
    import numpy as np  

    ### 從 Excel檔案讀取資料存成 dataframe ###
    df_basic = pd.read_excel(file_path, sheet_name='Basic', index_col=0, header=0)

    df_hotel_time = pd.read_excel(file_path, sheet_name='Hotel_Time', index_col=0, header=0)
    df_hotel_cost = pd.read_excel(file_path, sheet_name='Hotel_Cost', index_col=0, header=0)

    df_attraction_cost = pd.read_excel(file_path, sheet_name='Attraction_Cost', index_col=0, header=0)
    df_attrcation_open = pd.read_excel(file_path, sheet_name='Attraction_Open', index_col=0, header=0)
    df_attrcation_close = pd.read_excel(file_path, sheet_name='Attraction_Close', index_col=0, header=0)

    df_walk_time = pd.read_excel(file_path, sheet_name='Walk_Time', index_col=0, header=0)
    df_train_time = pd.read_excel(file_path, sheet_name='Train_Time', index_col=0, header=0)
    df_car_time = pd.read_excel(file_path, sheet_name='Car_Time', index_col=0, header=0)

    df_train_cost = pd.read_excel(file_path, sheet_name='Train_Cost', index_col=0, header=0)
    df_car_cost = pd.read_excel(file_path, sheet_name='Car_Cost', index_col=0, header=0)

    df_stay_happy = pd.read_excel(file_path, sheet_name='Stay_Happy', index_col=0, header=0)
    

    # BUDGET
    BUDGET = int(df_basic.iloc[0, 0])
    # M = len(Set of hotels and airpots)
    M = int(df_basic.iloc[1, 0])
    I = int(df_basic.iloc[2, 0])
    P = int(df_basic.iloc[3, 0])
    D = int(df_basic.iloc[4, 0])
    K = int(df_basic.iloc[5, 0])

    # 第一天抵達的時間
    # 在這個 instance0 中值是 540 = 540 / 60 = 早上 9:00
    Day1Arrival = df_basic.iloc[6, 0]
    print(Day1Arrival)

    LH = df_hotel_time.iloc[0:df_hotel_time.shape[0], 0]
    RH = df_hotel_time.iloc[0:df_hotel_time.shape[0], 1]
    

    STAY = df_stay_happy.iloc[0:df_stay_happy.shape[0], 0]  # stay time
    HAPPINESS = df_stay_happy.iloc[0:df_stay_happy.shape[0], 1]  # happiness

    ### 接下來會把不同交通方式所需的時間矩陣存到一個字典 DURATION 中 ###
    # 設一個變數存取不同地點到不同地點走路所需的時間矩陣
    walk_time = np.zeros((df_walk_time.shape[0], df_walk_time.shape[1]))
    for i in range(df_walk_time.shape[0]):
        for j in range(df_walk_time.shape[1]):
            walk_time[i][j] = df_walk_time.iloc[i, j]
    
    # 設一個變數存取不同地點到不同地點搭火車所需的時間矩陣
    train_time = np.zeros((df_train_time.shape[0], df_train_time.shape[1]))
    for i in range(df_train_time.shape[0]):
        for j in range(df_train_time.shape[1]):
            train_time[i][j] = df_train_time.iloc[i, j]
    
    # 設一個變數存取不同地點到不同地點搭計程車所需的時間矩陣
    car_time = np.zeros((df_car_time.shape[0], df_car_time.shape[1]))
    for i in range(df_car_time.shape[0]):
        for j in range(df_car_time.shape[1]):
            car_time[i][j] = df_car_time.iloc[i, j]

    DURATION = {
        'Walk': walk_time,
        'Train': train_time,
        'Car': car_time
    }

    ### 接下來會把不同交通方式所需的費用矩陣存到一個字典 C_TRAVEL 中 ###
    # 設一個變數存取不同地點到不同地點搭火車所需的花費矩陣
    walk_cost = np.zeros((df_train_cost.shape[0], df_train_cost.shape[1]))
    train_cost = np.zeros((df_train_cost.shape[0], df_train_cost.shape[1]))
    for i in range(df_train_cost.shape[0]):
        for j in range(df_train_cost.shape[1]):
            train_cost[i][j] = df_train_cost.iloc[i, j]

    # 設一個變數存取不同地點到不同地點搭計程車所需的花費矩陣
    car_cost = np.zeros((df_car_cost.shape[0], df_car_cost.shape[1]))
    for i in range(df_car_cost.shape[0]):
        for j in range(df_car_cost.shape[1]):
            car_cost[i][j] = df_car_cost.iloc[i, j]

    # 把兩種交通方式的花費矩陣包裝到一個字典中
    C_TRAVEL = {
        'Walk':walk_cost,
        'Train': train_cost,
        'Car': car_cost
    }


    C_HOTEL = np.zeros((df_hotel_cost.shape[0], df_hotel_cost.shape[1]))
    for i in range(df_hotel_cost.shape[0]):
        for j in range(df_hotel_cost.shape[1]):
            C_HOTEL[i][j] = df_hotel_cost.iloc[i, j]

    C_ATTRACTION = np.zeros(df_attraction_cost.shape[0])
    for i in range(df_attraction_cost.shape[0]):
            C_ATTRACTION[i] = df_attraction_cost.iloc[i]
            
    ATTRACTION_OPEN = np.zeros((df_attrcation_close.shape[0], df_attrcation_close.shape[1]))
    for i in range(df_attrcation_open.shape[0]):
        for j in range(df_attrcation_open.shape[1]):
            ATTRACTION_OPEN[i][j] = df_attrcation_open.iloc[i, j]

    ATTRACTION_CLOSE = np.zeros((df_attrcation_close.shape[0], df_attrcation_close.shape[1]))
    for i in range(df_attrcation_close.shape[0]):
        for j in range(df_attrcation_close.shape[1]):
            ATTRACTION_CLOSE[i][j] = df_attrcation_close.iloc[i, j]

    # 把參數包裝到一個字典中傳回
    results = {
        'BUDGET': BUDGET,
        'M': M,
        'I': I,
        'P': P,
        'D': D,
        'K': K,
        'LH': LH,
        'RH': RH,
        'Day1Arrival': Day1Arrival,
        'STAY': STAY,
        'HAPPINESS': HAPPINESS,
        'DURATION': DURATION,
        'C_TRAVEL': C_TRAVEL,
        'C_HOTEL': C_HOTEL,
        'C_ATTRACTION': C_ATTRACTION,
        'ATTRACTION_OPEN': ATTRACTION_OPEN,
        'ATTRACTION_CLOSE': ATTRACTION_CLOSE
    }
    
    return results


In [12]:
import numpy as np
# Read Excel file and load data
file_path = "Instances/Group1/Instance1_3.xlsx"

data = load_file(file_path)

budget = data['BUDGET']
num_days = data['D']
num_hotels = data['M']  
num_attractions = data['I']
num_places = data['P']
num_transport_modes = data['K']

# S_M = [m for m in range(1, data['M'] + 1)]  #  the first hotel is the starting airport and the last hotel is the leaving airport
# S_I = [i for i in range(1, data['I'] + 1)]
# S_P = [p for p in range(1, data['I'] + data['M'] + 1)]
# S_D = [d for d in range(1, data['D'] + 1)]
# S_K = [k for k in range(1, data['K'] + 1)]

S_M = [m for m in range(0, data['M'])]  #  the first hotel is the starting airport and the last hotel is the leaving airport
S_I = [i for i in range(0, data['I'])]
S_P = [p for p in range(0, data['I'] + data['M'])]
S_D = [d for d in range(0, data['D'])]
S_K = [k for k in range(0, data['K'])]




# 第一天抵達機場時間
arrival_time_day1 = data['Day1Arrival'] # 需要從 Excel 讀取

# 每天離開旅館的時間 (不包含第一天和最後一天)
departure_time_hotel = data['LH'] # 需要從 Excel 讀取
departure_time_hotel = departure_time_hotel.reset_index(drop=True)

# 每天返回旅館的最晚時間
return_time_hotel = data['RH']    # 需要從 Excel 讀取
return_time_hotel = return_time_hotel.reset_index(drop=True)

# 每個景點的停留時間 (表格，索引為景點 i)
stay_duration = data['STAY']      # 需要從 Excel 讀取 (DataFrame 或 dict)
stay_duration = stay_duration.reset_index(drop=True)

# 每個景點的成本 (表格，索引為景點 i)
spot_costs = data['C_ATTRACTION']         # 需要從 Excel 讀取 (DataFrame 或 dict)

# 每個景點的幸福值 (表格，索引為景點 i)
happiness_values = data['HAPPINESS']    # 需要從 Excel 讀取 (DataFrame 或 dict)
happiness_values = happiness_values.reset_index(drop=True)

# 每個景點每天的開放時間 (表格，索引為景點 i 和天 d)
open_times = data['ATTRACTION_OPEN']          # 需要從 Excel 讀取 (DataFrame 或 dict)

# 每個景點每天的關閉時間 (表格，索引為景點 i 和天 d)
close_times = data['ATTRACTION_CLOSE']         # 需要從 Excel 讀取 (DataFrame 或 dict)

# 旅館每天的住宿成本 (表格，索引為旅館 m 和天 d，不包含起始和結束機場)
hotel_costs = data['C_HOTEL']        # 需要從 Excel 讀取 (DataFrame 或 dict)

# 地點 p 到地點 q 使用交通方式 k 的交通時間 (表格，索引為 p, q, k)
transport_times = data['DURATION']      # 需要從 Excel 讀取 (DataFrame 或 MultiIndex DataFrame 或 dict)
transport_times = np.stack(list(transport_times.values()))
# 地點 p 到地點 q 使用交通方式 k 的交通成本 (表格，索引為 p, q, k)
transport_costs = data['C_TRAVEL']     # 需要從 Excel 讀取 (DataFrame 或 MultiIndex DataFrame 或 dict)
transport_costs = np.stack(list(transport_costs.values()))

#重要!補0
stay_duration = pd.concat([stay_duration, pd.Series(0 for _ in range(num_hotels))], ignore_index=True)

print(f"Budget: {budget}")
print(f"Number of days: {num_days}")
print(f"Number of hotels: {num_hotels}")
print(f"Number of attractions: {num_attractions}")
print(f"Number of places: {num_places}")
print(f"Number of transport modes: {num_transport_modes}")

print("Sets:")
print(f"S_M: {S_M}")
print(f"S_I: {S_I}")
print(f"S_P: {S_P}")
print(f"S_D: {S_D}")
print(f"S_K: {S_K}")

M = S_M
I = S_I
P = S_P
D = S_D
K = S_K


print(f"Day 1 Arrival Time: {arrival_time_day1}")
print(f"Departure Time from Hotel: {departure_time_hotel}")
print(f"Return Time to Hotel: {return_time_hotel}")
print(f"Stay Duration: {stay_duration}")
print(f"Spot Costs: {spot_costs}")
print(f"Happiness Values: {happiness_values}")
print(f"Open Times: {open_times}")
print(f"Close Times: {close_times}")
print(f"Hotel Costs: {hotel_costs}")
print(f"Transport Times: {transport_times}")
print(f"Transport Costs: {transport_costs}")

540
Budget: 100000
Number of days: 5
Number of hotels: 3
Number of attractions: 7
Number of places: 10
Number of transport modes: 3
Sets:
S_M: [0, 1, 2]
S_I: [0, 1, 2, 3, 4, 5, 6]
S_P: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
S_D: [0, 1, 2, 3, 4]
S_K: [0, 1, 2]
Day 1 Arrival Time: 540
Departure Time from Hotel: 0    540
1    600
2    540
3    540
4    660
Name: LH, dtype: int64
Return Time to Hotel: 0    1380
1    1380
2    1380
3    1380
4    1380
Name: RH, dtype: int64
Stay Duration: 0     60
1     30
2     45
3     40
4     30
5     90
6    120
7      0
8      0
9      0
dtype: int64
Spot Costs: [717.   0.   0. 696.   0.   0.   0.]
Happiness Values: 0    4.5
1    4.7
2    4.6
3    4.4
4    4.2
5    4.6
6    4.3
Name: Happiness, dtype: float64
Open Times: [[600. 600. 600. 600. 600.]
 [360. 360. 360. 360. 360.]
 [400. 400. 400. 400. 400.]
 [540. 540. 540. 540. 540.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]]
Close Times: [[1260. 1260. 1260. 1260. 1260.]

In [72]:
print(I)

[0, 1, 2, 3, 4, 5, 6]


In [73]:
departure_time_hotel

0    540
1    600
2    540
3    540
4    660
Name: LH, dtype: int64

In [74]:
open_times

array([[600., 600., 600., 600., 600.],
       [360., 360., 360., 360., 360.],
       [400., 400., 400., 400., 400.],
       [540., 540., 540., 540., 540.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.]])

In [75]:
print(D[1:-1])

[1, 2, 3]


In [76]:
spot_costs

array([717.,   0.,   0., 696.,   0.,   0.,   0.])

In [77]:
transport_costs

array([[[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.]],

       [[   0.,   38.,   58.,   55.,   38.,   50.,   75.,  220.,   55.,
          110.],
        [  38.,    

In [5]:
#重要!補0
stay_duration = pd.concat([stay_duration, pd.Series(0 for _ in range(num_hotels))], ignore_index=True)
stay_duration

0     60
1     30
2     45
3     40
4     30
5     90
6    120
7      0
8      0
9      0
dtype: int64

In [11]:
bigM = 10000
# ----- 3. 建立 Gurobi 模型 -----
import gurobipy as gp
from gurobipy import GRB
print("Creating model...")
model = gp.Model("travel_planning")
print("Model created.")

# ----- 4. 定義決策變數 -----
x = model.addVars(S_I, S_D, vtype=GRB.BINARY, name="visit")
t = model.addVars(S_P, S_P, S_K, S_D, vtype=GRB.BINARY, name="travel")
ha = model.addVars(S_M, S_D, vtype=GRB.BINARY, name="arrive_hotel")
hl = model.addVars(S_M, S_D, vtype=GRB.BINARY, name="leave_hotel")
a = model.addVars(S_P, S_D, vtype=GRB.CONTINUOUS, name="arrival_time")
print("Defined variables:")

# ----- 5. 設定目標函數 -----
# i 是從 num_hotels + 2 機場 + 1 開始算起，例如 instance0 中 i 從 4 開始
# 但是 happiness_values 等參數的 index 是從 0 開始算起
# 所以在 index 這邊就扣會回去
model.setObjective(gp.quicksum(happiness_values[i] * x[i, d] for d in S_D for i in S_I), GRB.MAXIMIZE)
print("Objective function set.")   


# ----- 6. 加入約束條件 -----
#預算約束 (equation 2)
model.addConstr(
    gp.quicksum(transport_costs[k,p,q] * t[p, q, k, d] for d in S_D for p in S_P for q in S_P if p != q for k in S_K)
    + gp.quicksum(hotel_costs[d,m] * ha[m, d] for m in S_M[1:-1] for d in S_D) 
    + gp.quicksum(spot_costs[i] * x[i, d] for d in S_D for i in S_I) <= budget,
    "budget_constraint"
    )
# 第一天抵達機場時間 (equation 3) delete
#model.addConstr(a[0, 0] == arrival_time_day1, "arrival_day1")

# 其他天離開旅館時間 (equation 4) delete
# for d in D[1:]:
#     for m in M[1:-1]:
#         model.addConstr(a[m, d] == departure_time_hotel[d] * hl[m, d], f"departure_hotel_{m}_{d}")

# 抵達景點時間下限 (equation 5)
for i in I:
    for d in D:
        model.addConstr(a[i, d] >= departure_time_hotel[d] * x[i, d], f"arrival_spot_lower_{i}_{d}")

# 景點開放時間約束 (equation 6)
for i in I:
    for d in D:
        model.addConstr(a[i, d] >= open_times[i,d] * x[i, d], f"open_time_{i}_{d}")

# 定義一個足夠大的時間值，用於取代無限大 (Infinity)
# 假設您的時間單位是小時，一天有 24 小時，最多5天，那麼 1000 是一個保守的足夠大的數
MAX_POSSIBLE_TIME = 10000.0 # 例如，用 10000 分鐘代表"無限大"

#景點關閉時間約束 (equation 7)
for i in I:
    for d in D:
        # 將 float('inf') 替換為 MAX_POSSIBLE_TIME
        model.addConstr(a[i, d] + stay_duration[i] * x[i, d] <= close_times[i,d] * x[i, d], f"close_time_{i}_{d}")

# --- 新增的住宿地點相關約束 ---

# 不可以跑回去睡機場 (離開)
# h^l_{1d} = 0, forall d in D \setminus \{1\}
for d in D:
    if d != 0:
        model.addConstr(hl[0, d] == 0, f"no_leave_start_airport_{d}")

# 不可以跑回去睡機場 (抵達)
# h^a_{n_md} = 0, forall d in D \setminus \{n_d\}
for d in D:
    if d != num_days - 1: # num_days 是您之前定義的總天數
        model.addConstr(ha[num_hotels - 1, d] == 0, f"no_arrive_end_airport_{d}")


# ... (您後續的其他約束，例如每天只能住一間旅館等)

# 第一天返回旅館時間約束 (equation 8)
model.addConstr(
    arrival_time_day1 +
    gp.quicksum(stay_duration[i] * x[i, 0] for i in I) +
    gp.quicksum(transport_times[k,p,q] * t[p, q, k, 0] for p in P for q in P if p != q for k in K) <= return_time_hotel[0],
    "return_day1"
)

# 其他天返回旅館時間約束 (equation 9)
for d in D[1:]:
    model.addConstr(
        departure_time_hotel[d] + 
        gp.quicksum(stay_duration[i] * x[i, d] for i in I) +
        gp.quicksum(transport_times[k,p,q] * t[p, q, k, d] for p in P for q in P if p != q for k in K) <= return_time_hotel[d],
        f"return_day_{d}"
    )

# 同一天內地點間移動的時間約束 (equation 10)

#從hotel 出發 (new equation)
for d in D:
    for q in P:
        travel_time_expr = gp.quicksum(transport_times[k,num_attractions+m,q] * t[num_attractions+m, q, k, d] for k in K for m in M)
        lhs = departure_time_hotel[d]  + travel_time_expr - a[q, d]
        rhs = bigM * (1 - gp.quicksum(t[num_attractions+m, q, k, d] for k in K for m in M))
        #print(f"p=hotel, q={q}, d={d}, lhs={lhs}, rhs={rhs}") # <--- 在這裡添加 print 語句
        model.addConstr(lhs <= rhs, f"travel_time_hotel_{q}_{d}")

#from tourist spots to other places (updated equation 10)
for i in I:
    for q in P:
        if i != q:
            for d in D:
                travel_time_expr = gp.quicksum(transport_times[k,i,q] * t[i, q, k, d] for k in K)
                lhs = a[i, d] + stay_duration[i] + travel_time_expr - a[q, d]
                rhs = bigM * (1 - gp.quicksum(t[i, q, k, d] for k in K))
                #print(f"p={i}, q={q}, d={d}, lhs={lhs}, rhs={rhs}") # <--- 在這裡添加 print 語句
                model.addConstr(lhs <= rhs, f"travel_time_{i}_{q}_{d}")


# 每個景點最多參觀一次 (equation 11)
for i in I:
    model.addConstr(gp.quicksum(x[i, d] for d in D) <= 1, f"visit_once_{i}")

# 每天必須抵達一個旅館(equation 12)
for d in D:
    model.addConstr(gp.quicksum(ha[m, d] for m in M) == 1, f"arrive_one_hotel_{d}")

# 如果今晚抵達旅館 m，明天早上必須從同一旅館 m 出發(最後一天除外) (equation 13)
for m in M:
    for d in D[:-1]:
        model.addConstr(ha[m, d] == hl[m, d + 1], f"hotel_continuity_{m}_{d}")

# 第一天早上必須離開機場 (equation 14)
model.addConstr(hl[0, 0] == 1, "leave_airport_day1")

# 最後一天晚上必須抵達機場 (equation 15)
model.addConstr(ha[num_hotels - 1, num_days - 1] == 1, "arrive_airport_last_day")

# 離開景點後的去向 (equation 16) - 修改為逐日約束
for i in I:
    for d in D: # 新增 d 的外層循環
        model.addConstr(
            gp.quicksum(t[i, j, k, d] for j in I for k in K if i != j) + 
            gp.quicksum(t[i, num_attractions + m, k, d] for m in M for k in K) == x[i, d], 
            f"leave_spot_{i}_{d}" # 修改約束名稱以包含 d
        )

# 前往景點的起點 (equation 17) - 修改為逐日約束
for i in I:
    for d in D: # 新增 d 的外層循環
        model.addConstr(
            gp.quicksum(t[j, i, k, d] for j in I for k in K if i != j) + # 移除 d 的循環
            gp.quicksum(t[num_attractions + m, i, k, d] for m in M for k in K) == x[i, d], # 移除 d 的循環
            f"arrive_spot_{i}_{d}" # 修改約束名稱以包含 d
        )

# 早上離開旅館後的去向 (equation 18)
for m in M:
    for d in D:
        model.addConstr(
            gp.quicksum(t[num_attractions + m, q, k, d] for q in P if q != (num_attractions + m) for k in K) == hl[m, d],
            f"leave_hotel_to_spot_{m}_{d}"
        )

# 晚上離開景點後要去旅館 (equation 19)
for m in M:
    for d in D:
        model.addConstr(
            gp.quicksum(t[j, num_attractions + m, k, d] for j in I for k in K) == ha[m, d],
            f"arrive_hotel_from_spot_{m}_{d}"
        )

# 每個地方一天只能到達一次 (new equation)
for d in D:
    for q in P:
        model.addConstr(
            gp.quicksum(t[p, q, k, d] for p in P if p != q for k in K) <= 1,
            f"arrive_maximumn_once_at_each_spot{q}_on_each_day_{d}"
        )

# 每個地方只能用一種交通方式抵達 (new equation)
for d in D:
    for p in P:
        for q in P:
            model.addConstr(
                gp.quicksum(t[p, q, k, d] for k in K) <= 1,
                f"one transportation type{p}_{q}_{d}"
            )


# ----- 7. 求解模型 -----
model.optimize()

Creating model...
Model created.
Defined variables:
Objective function set.
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.4.0 24E263)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1160 rows, 1615 columns and 10214 nonzeros
Model fingerprint: 0xb38e2a66
Variable types: 50 continuous, 1565 integer (1565 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 1148 rows and 1581 columns
Presolve time: 0.01s
Presolved: 12 rows, 34 columns, 68 nonzeros
Variable types: 0 continuous, 34 integer (34 binary)
Found heuristic solution: objective 22.0000000
Found heuristic solution: objective 22.2000000
Found heuristic solution: objective 22.4000000

Root relaxation: objective 2.280000e+01, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    | 

In [80]:
# ----- 8. 輸出結果 -----
if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:") # 這是新增的
    print("--- Arrival Times (a[p, d]) ---") # 這是新增的
    for p in P: # 這是新增的
        for d in D: # 這是新增的
            if a[p,d].X is not None: # 這是新增的
                print(f"Arrival time at location {p} on day {d}: {a[p,d].X:.2f}") # 這是新增的
    print("---------------------------------") # 這是新增的

    # 您也可以選擇性地打印其他變數的值，例如 x[i,d] (表示景點是否被訪問)
    print("\n--- Visited Spots (x[i, d]) ---") # 這是新增的
    for i in I: # 這是新增的
        for d in D: # 這是新增的
            if x[i,d].X > 0.5: # 這是新增的
                print(f"Spot {i} visited on day {d}") # 這是新增的
    print("-------------------------------") # 這是新增的

    # 如果您還想看看交通方式 t 的選擇
    print("\n--- Chosen Travel Modes (t[p, q, k, d]) ---") # 這是新增的
    for p in P: # 這是新增的
        for q in P: # 這是新增的
            if p != q: # 這是新增的
                for d in D: # 這是新增的
                    for k in K: # 這是新增的
                        if t[p, q, k, d].X > 0.5: # 這是新增的
                            print(f"Travel from {p} to {q} using mode {k+1 if K else 1} on day {d}") # 這是新增的
    print("---------------------------------------------") # 這是新增的
    print("\n最佳化完成！")
    print(f"最大總幸福值：{model.objVal}")

    
# 輸出哪些景點在哪天參觀
    for d in D:
        print(f"\n第 {d} 天：")
        for i in I:
            if x[i, d].x > 0.5:
                print(f"  參觀景點 {i}")

    # 輸出每天的行程 (包含移動)
    for d in D:
        print(f"\n第 {d} 天詳細行程：")
        current_location = num_attractions if d == 0 else None # 第一天從機場開始
        leave_hotel = num_attractions if d == 0 else None
        if d >= 1:
            for m in M:
                if hl[m, d].x > 0.5:
                    leave_hotel = num_attractions+m
                    current_location = num_attractions+m
                    print(f"離開旅館 {m} (時間: {departure_time_hotel[d]})")
                    break

        while True:
            moved = False
            for p in P:
                if p == current_location:
                    for q in P:
                        if p != q:
                            for k in K:
                                if t[p, q, k, d].x > 0.5:
                                    print(f"  從地點 {p} 移動到地點 {q} (交通方式 {k+1 if K else 1}, 花費時間: {transport_times[k][p][q]})")
                                    current_location = q
                                    if q in I and x[q, d].x > 0.5:
                                        print(f"  參觀景點 {q} (抵達時間: {a[q, d].x}, 停留時間: {stay_duration[q]})")
                                    moved = True
                                    break
                            if moved:
                                break
                    break
            if not moved or current_location == leave_hotel:
                # 檢查是否抵達旅館
                for m in M:
                    if ha[m, d].x > 0.5:
                        print(f" 抵達地點 {num_attractions+m} 也就是旅館 {m} (時間: {a[num_attractions+m, d].x})")
                        current_location = num_attractions + m
                        moved = True
                        break
                if moved:
                    break # 當天沒有更多移動

elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible (無可行解)")
    print("Consider checking constraints for conflicts.")
    model.computeIIS()  # <--- 取消註解這一行
    model.write("infeasible.ilp") # <--- 取消註解這一行
    print("Infeasible constraints written to infeasible.ilp")
    # 如果您還想將模型保存為LP格式以供其他工具查看 (可選)
    # model.write("full_model.lp")

else:
    print("模型沒有找到最佳解")


Optimal solution found:
--- Arrival Times (a[p, d]) ---
Arrival time at location 0 on day 0: 0.00
Arrival time at location 0 on day 1: 0.00
Arrival time at location 0 on day 2: 0.00
Arrival time at location 0 on day 3: 840.00
Arrival time at location 0 on day 4: 0.00
Arrival time at location 1 on day 0: 0.00
Arrival time at location 1 on day 1: 0.00
Arrival time at location 1 on day 2: 840.00
Arrival time at location 1 on day 3: -0.00
Arrival time at location 1 on day 4: 0.00
Arrival time at location 2 on day 0: 0.00
Arrival time at location 2 on day 1: 900.00
Arrival time at location 2 on day 2: 0.00
Arrival time at location 2 on day 3: -0.00
Arrival time at location 2 on day 4: 0.00
Arrival time at location 3 on day 0: 840.00
Arrival time at location 3 on day 1: 0.00
Arrival time at location 3 on day 2: 0.00
Arrival time at location 3 on day 3: -0.00
Arrival time at location 3 on day 4: -0.00
Arrival time at location 4 on day 0: -0.00
Arrival time at location 4 on day 1: 0.00
Arriva

In [81]:
departure_time_hotel

0    540
1    600
2    540
3    540
4    660
Name: LH, dtype: int64

In [82]:
print(budget)

100000


In [83]:
type(hotel_costs)

numpy.ndarray